In [1]:
%env MKL_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env OMP_NUM_THREADS=1

env: MKL_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1


In [2]:
import multiprocessing
import concurrent.futures as cf

import matplotlib.pyplot as plt
import numpy as np
import math
import pybobyqa

import cirq
import sympy as sp

In [3]:
#Qubits on Square-Octagon Lattice w/ Lattice Parameters
qubits = [0, 1, 2, 3, 4, 5, 6, 7]
Nq = len(qubits)

xx_links = [(0, 1), (2, 3)]
yy_links = [(0, 3), (1, 2)]
yy_links_reversed = [(1, 2), (0, 3)]
zz_links = [(0, 4), (1, 5), (2, 6), (3, 7)]
zz_links_reversed = [(3, 7), (2, 6), (1, 5), (0, 4)]

Jx, Jy, Jz = -1.0/math.sqrt(2), -1.0/math.sqrt(2), -1.0
hX = hY = hZ = 0.05/math.sqrt(3)

In [4]:
#Hamiltonian Construction

#Qubit register for Hamiltonian
qreg = cirq.NamedQubit.range(Nq,prefix="q")

#Constructing Pauli Strings
XX_links_strings = []
for (i,j) in xx_links:
    XX_string = []
    for q in range(Nq):
        if q != i and q != j:
            XX_string.append(cirq.I(qreg[q]))
        else:
            XX_string.append(cirq.X(qreg[q]))
    XX_links_strings.append(XX_string)
    
YY_links_strings = []
for (i,j) in yy_links:
    YY_string = []
    for q in range(Nq):
        if q != i and q != j:
            YY_string.append(cirq.I(qreg[q]))
        else:
            YY_string.append(cirq.Y(qreg[q]))
    YY_links_strings.append(YY_string)

ZZ_links_strings = []
for (i,j) in zz_links:
    ZZ_string = []
    for q in range(Nq):
        if q != i and q != j:
            ZZ_string.append(cirq.I(qreg[q]))
        else:
            ZZ_string.append(cirq.Z(qreg[q]))
    ZZ_links_strings.append(ZZ_string)

X_qubit_strings = []
Y_qubit_strings = []
Z_qubit_strings = []
for k in qubits:
    X_string = []
    Y_string = []
    Z_string = []
    for q in range(Nq):
        if q != k:
            X_string.append(cirq.I(qreg[q]))
            Y_string.append(cirq.I(qreg[q]))
            Z_string.append(cirq.I(qreg[q]))
        else:
            X_string.append(cirq.X(qreg[q]))
            Y_string.append(cirq.Y(qreg[q]))
            Z_string.append(cirq.Z(qreg[q]))
    X_qubit_strings.append(X_string)
    Y_qubit_strings.append(Y_string)
    Z_qubit_strings.append(Z_string)

In [5]:
#Hamiltonian Construction (ctd.)

#Convert Pauli Strings into Products
XX_links_terms = []
for XX_link in XX_links_strings:
    XX_links_terms.append(cirq.PauliString(XX_link))

YY_links_terms = []
for YY_link in YY_links_strings:
    YY_links_terms.append(cirq.PauliString(YY_link))
    
ZZ_links_terms = []
for ZZ_link in ZZ_links_strings:
    ZZ_links_terms.append(cirq.PauliString(ZZ_link))

X_qubit_terms = []
for qubit_string in X_qubit_strings:
    X_qubit_terms.append(cirq.PauliString(qubit_string))

Y_qubit_terms = []
for qubit_string in Y_qubit_strings:
    Y_qubit_terms.append(cirq.PauliString(qubit_string))

Z_qubit_terms = []
for qubit_string in Z_qubit_strings:
    Z_qubit_terms.append(cirq.PauliString(qubit_string))

In [6]:
#Hamiltonian Construction (ctd.)

#Add up all the terms
Hamiltonian = 0
for XX_link in XX_links_terms:
    Hamiltonian += Jx*XX_link
for YY_link in YY_links_terms:
    Hamiltonian += Jy*YY_link
for ZZ_link in ZZ_links_terms:
    Hamiltonian += Jz*ZZ_link
for X_qubit in X_qubit_terms:
    Hamiltonian += hX*X_qubit
for Y_qubit in Y_qubit_terms:
    Hamiltonian += hY*Y_qubit
for Z_qubit in Z_qubit_terms:
    Hamiltonian += hZ*Z_qubit

In [7]:
##Exact Diagonalization of Hamiltonian (Square-Octagon Model)
eigvals, eigvecs = np.linalg.eig(Hamiltonian.matrix())
print(f"Min eigvalue: {min(eigvals)}")

Min eigvalue: (-4.701075608032761-3.958045905044745e-17j)


In [8]:
##Hamiltonian Variational Ansatz Circuit Construction
def HVA_circuit(alpha, beta, gamma, alpha_mag, beta_mag, gamma_mag, n_layers):
    global xx_links
    global yy_links
    global yy_links_reversed
    global zz_links
    global zz_links_reversed
    global qubits
    #Initialize Circuit
    HVA = cirq.Circuit()
    qreg_HVA = cirq.NamedQubit.range(Nq, prefix="q")
    #Declare variational parameters
    #alpha = sp.symarray('alpha', n_layers)
    #beta = sp.symarray('beta', n_layers)
    #gamma = sp.symarray('gamma', n_layers)
    #alpha_mag = sp.symarray('alpha_mag', n_layers)
    #beta_mag = sp.symarray('beta_mag', n_layers)
    #gamma_mag = sp.symarray('gamma_mag', n_layers)
    #Ansatz Construction
    for idx_layer in range(n_layers):
    #Construct X-terms portion of layer
        HVA.append(cirq.rx(2*alpha_mag[idx_layer]).on(q) for q in qreg_HVA)
        for (i,j) in xx_links:
            HVA.append(cirq.H(qreg_HVA[i]))
            HVA.append(cirq.H(qreg_HVA[j]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
            HVA.append(cirq.rz(2*alpha[idx_layer]).on(qreg_HVA[min(i,j)]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
            HVA.append(cirq.H(qreg_HVA[i]))
            HVA.append(cirq.H(qreg_HVA[j]))
    #Construct Y-terms portion of layer
        HVA.append(cirq.ry(2*beta_mag[idx_layer]).on(q) for q in qreg_HVA)
        for (i,j) in yy_links:
            HVA.append(cirq.rx(math.pi/2).on(qreg_HVA[i]))
            HVA.append(cirq.rx(math.pi/2).on(qreg_HVA[j]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
        for (i,j) in yy_links_reversed:
            HVA.append(cirq.rz(2*beta[idx_layer]).on(qreg_HVA[min(i,j)]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
            HVA.append(cirq.rx(-math.pi/2).on(qreg_HVA[i]))
            HVA.append(cirq.rx(-math.pi/2).on(qreg_HVA[j]))
    #Construct Z-terms portion of layer
        HVA.append(cirq.rz(2*gamma_mag[idx_layer]).on(q) for q in qreg_HVA)
        for (i,j) in zz_links:
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
        for (i,j) in zz_links_reversed:
            HVA.append(cirq.rz(2*gamma[idx_layer]).on(qreg_HVA[min(i,j)]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
    return HVA

In [9]:
##Construct Objective Function
def objective(params, hamiltonian, qreg, simulator):
    alpha = params[:n_layers]
    beta = params[n_layers:2*n_layers]
    gamma = params[2*n_layers:3*n_layers]
    alpha_mag = params[3*n_layers:4*n_layers]
    beta_mag = params[4*n_layers:5*n_layers]
    gamma_mag = params[5*n_layers:]
    ansatz = HVA_circuit(alpha, beta, gamma, alpha_mag, beta_mag, gamma_mag, n_layers)
    #expect = simulator.simulate_expectation_values(ansatz, hamiltonian)
    psi = ansatz.final_state_vector()
    expect = hamiltonian.expectation_from_state_vector(psi, qubit_map={q: i for i, q in enumerate(qreg)})
    return np.real(expect)

In [10]:
##Construct Objective Function
def objective_modified(params, hamiltonian_matrix, n_layers):
    simulator = cirq.Simulator()
    qreg = cirq.NamedQubit.range(Nq, prefix="q")

    alpha = params[:n_layers]
    beta = params[n_layers:2*n_layers]
    gamma = params[2*n_layers:3*n_layers]
    alpha_mag = params[3*n_layers:4*n_layers]
    beta_mag = params[4*n_layers:5*n_layers]
    gamma_mag = params[5*n_layers:]
    ansatz = HVA_circuit(alpha, beta, gamma, alpha_mag, beta_mag, gamma_mag, n_layers)
    #expect = simulator.simulate_expectation_values(ansatz, hamiltonian)
    psi = ansatz.final_state_vector()
    expect = hamiltonian_matrix.dot(psi)
    expect = psi.conjugate().dot(expect)
    return np.real(expect)

In [11]:
n_layers = 4
paras_random = np.random.uniform(-np.pi, np.pi, size = (6*n_layers, ))
simulator = cirq.Simulator()
qreg = cirq.NamedQubit.range(Nq, prefix="q")
hamiltonian_matrix = Hamiltonian.matrix()

print(objective(paras_random, Hamiltonian, qreg, simulator))
print(objective_modified(paras_random, hamiltonian_matrix, n_layers))

-1.1557166222757882
-1.155716622275788


In [12]:
%%time
n_layers = 4
paras_random = np.random.uniform(-np.pi, np.pi, size = (6*n_layers, ))
hamiltonian_matrix = Hamiltonian.matrix()

for j in range(200):
    objective_modified(paras_random, hamiltonian_matrix, n_layers)

CPU times: user 7.68 s, sys: 31.6 ms, total: 7.71 s
Wall time: 7.72 s


In [13]:
%%time
n_layers = 4
paras_random = np.random.uniform(-np.pi, np.pi, size = (6*n_layers, ))
hamiltonian_matrix = Hamiltonian.matrix()

ex = cf.ProcessPoolExecutor(max_workers=10)
job_list = [ex.submit(objective_modified, paras_random, hamiltonian_matrix, n_layers) for j in range(200)]
ex.shutdown(wait=True)
result = [job.result() for job in job_list]

CPU times: user 199 ms, sys: 119 ms, total: 318 ms
Wall time: 1.02 s


In [14]:
##Define a function that executes VQE for parallelization purposes 
def VQE(initial_params, n_layers, cf_evals_max, ham, qreg, sim):
    res = pybobyqa.solve(objective, x0=initial_params, args=(ham, qreg, sim), maxfun = cf_evals_max)
    return res

In [15]:
##Define a function that executes VQE for parallelization purposes 
def VQE_modified(initial_params, n_layers, cf_evals_max, hamiltonian_matrix):
    res = pybobyqa.solve(
        objective_modified, x0=initial_params,
        args=(hamiltonian_matrix, n_layers), maxfun = cf_evals_max)
    return res

In [17]:
%%time
##Perform VQE

#Initialize the simulator
simulator = cirq.Simulator()
qubit_reg = cirq.NamedQubit.range(Nq, prefix="q")

#Specify number of workers to use in parallelization (use maximum number of cores)
n_workers_use = 64

#Specify max number of layers
n_layers_max = 1

#Specify number of initial values
n_initial_vals_max = 501

#Specify max number of cost function evaluations
cost_function_evals_max = 2000

#Loop that executes VQE
for n_layers in range(n_layers_max, n_layers_max+1):
    initial_values = np.random.uniform(-np.pi, np.pi, size = (n_initial_vals_max, 6*n_layers))
    ex = cf.ProcessPoolExecutor(max_workers=n_workers_use)
    job_list = [ex.submit(VQE_modified, initial_value, n_layers, cost_function_evals_max, hamiltonian_matrix) for initial_value in initial_values]
    ex.shutdown(wait=True)
    result = [job.result() for job in job_list]

CPU times: user 685 ms, sys: 578 ms, total: 1.26 s
Wall time: 1min 4s


In [10]:
#for n_layers in range(n_layers_max):
    #print(f"No. of layers: {n_layers+1}")
    #print(f"Optimal parameters: {list(res_min[n_layers].x)}")
    #print(f"Optimal value: {res_min[n_layers].f}")
    #print("*" * 50)

No. of layers: 1
Optimal parameters: [6.132852122445387, 2.991268338774197, 3.1392684872003183, 4.713424896582486, -0.02710765361176536, 8.24423078313647]
Optimal value: -4.678564702339708
**************************************************
No. of layers: 2
Optimal parameters: [3.298782136001315, -0.03302327241939069, 3.1462213653946804, 3.2954807146277787, 3.149245464880128, 4.740751429728666, 4.1192833100245245, 3.589762258880153, 1.9187015625173822, 5.289558018160561, -1.0683487418117092, 4.327524726622344]
Optimal value: -4.6823336259459865
**************************************************
No. of layers: 3
Optimal parameters: [1.4156131955489493, 3.449826663118241, 1.560899668175568, 2.3765484022897243, 1.7672767186153102, 2.9346387246414083, 6.271535044368608, 6.283133998440727, 1.5546148892715566, 3.8891612676792895, 2.147826786055702, 3.351377011687746, 1.437976038732945, 1.6686666301242685, 2.380734934399784, 2.233801896628324, 1.64008168270787, 2.2293756016476167]
Optimal valu